In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
def test_func():
    print("it works!! :o)")

In [ ]:
def is_question(row):
    return row.find('?')

In [ ]:
swear_words = pd.read_csv('list_swear.txt', delimiter = "\n")
swear_words_set = set(swear_words['4r5e'])
swear_words_set.add('4r5e')

def swear_o_meter_detection(row):
    swear_o_meter=0
#     print(row)
    text = row.split()
    for i in text:
        if i in swear_words_set:
            swear_o_meter += 1
    
    return swear_o_meter

In [ ]:

def explain_text_entities(data):
    
    temp  = concat_all_text(data)
    doc = nlp(temp)
    entities = []
    
#     ent_pd = []
    
#     for ent in doc.ents:
#         ent_pd.append[ent.text, ent.start_char, ent.end_char, ent.label_]
#         print(ent.text, ent.start_char, ent.end_char, ent.label_)
    
    
    for ent in doc.ents:
        entities.append([ent.text, ent.start_char, ent.end_char, ent.label_])
#         entities.extend([{ent}, {ent.label_}, {spacy.explain(ent.label_)}])
#         entities.extend(f'Entity: {ent}, Label: {ent.label_}, {spacy.explain(ent.label_)}')
#         print(f'Entity: {ent}, Label: {ent.label_}, {spacy.explain(ent.label_)}')
    return entities

In [1]:
def get_stop_words(type="stopwordslib"):
    more_stopwords = set(['t','dtypes','n','dtype','v','wo','s','object'])
    if(type=="stopwordslib"):      
        from nltk.corpus import stopwords
        my_stop = stopwords.words('english')
#         my_stop = my_stop.union( more_stopwords)
    elif(type=="from_wordcloud"):
        my_stop =  set(STOPWORDS)
        my_stop = my_stop.union( more_stopwords)
    
    return my_stop

In [1]:
### Import Statements ###

# Graphs section

import seaborn as sn
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Imports for wordcloud
# from subprocess import check_output
# from wordcloud import WordCloud, STOPWORDS

# Python spellchecker
# !pip install pyspellchecker
# from spellchecker import SpellChecker
# import string
# Textblob -> Taken from tina's notebook mentioned above :o)
!pip install TextBlob
from textblob import TextBlob
from textblob import Word

# For regular expressions
import re

In [15]:


# Regex 'Mask' to filter out garbage characters and stuff
regex = r"[^0-9a-zA-Z\']+"

def clean_lines(row):
    subst = " "
    row = re.sub(regex,subst,row,0)
    return row

In [16]:
# Function to concat all text from a column into a single string
def concat_all_text(data_col):
    text = data_col.str.cat(sep=' ')

    # removing URLs and '&amp' substrings using regex
    import re
    url_reg  = r'[a-z]*[:.]+\S+'
    text   = re.sub(url_reg, '', text)
    noise_reg = r'\&amp'
    text   = re.sub(noise_reg, '', text)
    regex = r"[^0-9a-zA-Z\']+"
    subst = " "
    text = re.sub(regex,subst,text,0)
    
    return text

In [17]:
# Thanks to thte following kaggler for this notebook! ->  https://www.kaggle.com/shahules/basic-eda-cleaning-and-glove/notebook#Data-Cleaning
# Section containing various "text cleaning" functions -> WIP!
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)


def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)


def full_clean(text):
    text = remove_punct(text)
    text = remove_html(text)
    text = remove_URL(text)
    text = remove_emoji(text)
    return text


In [18]:
# Pipeline to process a full column from a dataset
def processRow(row):
    
    # Simple porcessing stuff (lowercasing, removing emoji,URL etc ...)
    temp = row.lower()
    temp = remove_emoji(temp)
    temp = remove_URL(temp)
    temp = remove_html(temp)
    temp = re.sub(r'[^\w\s]'," ",temp)
    
    # remove numbers
    temp = "".join([i for i in temp if not i.isdigit()])
    
    # Expand contraction ('im' -> 'i am' )
    temp = expand_contractions(temp)
    # Lemmatize & remove stopwords
    from nltk.corpus import stopwords
    my_stop = set(stopwords.words('english'))
    temp = " ".join([Word(word).lemmatize() for word in temp.split() if word not in my_stop if len(word)>1])
    
    # Return finalized row
    return temp

# Function to get the frequencies 
def get_fequencies(doc_column):
    doc_column = concat_all_text(doc_column)
    tokens = nltk.word_tokenize(doc_column)
    frequency_dist = nltk.FreqDist(tokens)

    import operator
    sorted_x = sorted(frequency_dist.items(), key=operator.itemgetter(1),reverse=True)
    sorted_x = sorted_x[:100]
    return sorted_x


In [19]:
# Note: Need to find back the kaggle notebook from which I find how to do this! 
def print_wordcloud(text_data,stop_word=None,max_words=200):
    plt.rcParams['figure.figsize']=(15,9)
    plt.rcParams['font.size']=12                #10 
    plt.rcParams['savefig.dpi']=100             #72 
    plt.rcParams['figure.subplot.bottom']=.1 

    if(stop_word == None):
        print("Is none")
        stopwords = set(STOPWORDS)
    else:
        print("Custom stop word sent to function")
        stopwords = stop_word
        
    wordcloud = WordCloud(
                              background_color='white',
                              stopwords=stopwords,
                              max_words=max_words,
                              max_font_size=40, 
                              random_state=42
                             ).generate(str(text_data))

    print(wordcloud)
    fig = plt.figure(1)
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()
    fig.savefig("word1.png", dpi=900)
    
    return wordcloud

# %% [code] {"jupyter":{"outputs_hidden":false}}
print("all done?!")

all done?!


In [20]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}


def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def tsnescatterplot(model, word, list_names):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 300), dtype='f')
    word_labels = [word]
    color_list  = ['red']

    # adds the vector of the query word
    arrays = np.append(arrays, model.wv.__getitem__([word]), axis=0)
#     print(arrays)
    # gets list of most similar words
    close_words = model.wv.most_similar([word])
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = model.wv.__getitem__([wrd])
        word_labels.append(wrd)
        color_list.append('green')
        arrays = np.append(arrays, wrd_vector, axis=0)
        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=21
                ,svd_solver='full').fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title('t-SNE visualization for {}'.format(word.title()))

In [ ]:
# import multiprocessing

# from gensim.models import Word2Vec
# cores = multiprocessing.cpu_count()

# from gensim.models.phrases import Phrases, Phraser
# sent = [row.split() for row in data_kenjee_corr[data_kenjee_corr['is_question' >-1]]['clean_comment']]
# phrases = Phrases(sent, min_count=30, progress_per=10000)
# bigram = Phraser(phrases)

# sentences = bigram[sent]

# w2v_model = Word2Vec(min_count=20,
#                      window=2,
#                      vector_size=300,
#                      sample=6e-5, 
#                      alpha=0.03, 
#                      min_alpha=0.0007, 
#                      negative=20,
#                      workers=cores-1)
# w2v_model.build_vocab(sentences, progress_per=10000)